In [2]:
import librosa
import librosa.display
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np



Using TensorFlow backend.


In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import os
import tensorflow as tf

In [4]:
DATA_PATH = 'C:/Users/ppeng/Documents/AI_babyclassified/detecting/'

In [5]:
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)


In [6]:
labels, _, _ = get_labels(DATA_PATH)
data = {}
for label in labels:
    
    data[label] = {}
    data[label]['path'] = [DATA_PATH  + label + '/' +  wavfile for wavfile in os.listdir(DATA_PATH + '/' + label)]
    D = []
    for wavfile in data[label]['path']:
        y, sr = librosa.load(wavfile , duration = 4.97)  
        ps = librosa.feature.melspectrogram(y=y, sr=sr)
        D.append(ps)
    data[label]['melspec'] = D
print(labels)
print(D)

['animal', 'babycry', 'exterior', 'Interiordomestic sounds']
[array([[1.00494881e+01, 2.27782917e+01, 2.11724567e+01, ...,
        1.40596863e+02, 2.13456512e+02, 1.31084351e+02],
       [7.63773298e+00, 8.49055004e+00, 1.08542643e+01, ...,
        1.25518967e+02, 2.35057693e+02, 8.25480957e+01],
       [2.40505104e+01, 3.04308243e+01, 6.93421021e+01, ...,
        5.02066193e+01, 1.03028885e+02, 9.97923965e+01],
       ...,
       [1.98471639e-03, 1.38558983e-03, 9.27569985e-04, ...,
        1.71762466e+00, 1.78382254e+00, 2.04101801e+00],
       [1.67087663e-03, 8.47175543e-04, 1.90027320e-04, ...,
        1.35063212e-02, 1.90369915e-02, 4.57671434e-02],
       [2.06940647e-04, 1.13773283e-04, 2.14210922e-05, ...,
        4.08177584e-04, 2.23676814e-03, 8.01561680e-03]], dtype=float32), array([[8.59935486e+02, 4.59129913e+02, 2.63542206e+02, ...,
        3.27254715e+01, 2.46990128e+01, 1.73853931e+01],
       [5.71426125e+01, 2.95745678e+01, 2.27645569e+01, ...,
        1.21465073e+01

In [7]:
print(labels)
print(len(labels))

['animal', 'babycry', 'exterior', 'Interiordomestic sounds']
4


In [8]:
Data = []
for key in data:
      for melspec in data[key]['melspec']:
            Data.append((key,melspec))
data[key]['melspec'] = Data
print("Number of samples: ", len(Data))

Number of samples:  1641


In [80]:
import random
print("Number of samples: ", len(Data))
random.shuffle(Data)

train = Data[:1200]
test = Data[1200:]

y_train,X_train = zip(*train)
y_test, X_test = zip(*test)
X_train= np.array(X_train)
X_test = np.array(X_test)

X_train.shape
y_train

Number of samples:  1641


('babycry',
 'Interiordomestic sounds',
 'Interiordomestic sounds',
 'exterior',
 'Interiordomestic sounds',
 'exterior',
 'Interiordomestic sounds',
 'exterior',
 'babycry',
 'Interiordomestic sounds',
 'Interiordomestic sounds',
 'exterior',
 'animal',
 'exterior',
 'babycry',
 'animal',
 'exterior',
 'animal',
 'Interiordomestic sounds',
 'animal',
 'exterior',
 'Interiordomestic sounds',
 'exterior',
 'babycry',
 'babycry',
 'exterior',
 'animal',
 'Interiordomestic sounds',
 'Interiordomestic sounds',
 'exterior',
 'babycry',
 'animal',
 'exterior',
 'babycry',
 'exterior',
 'animal',
 'exterior',
 'exterior',
 'Interiordomestic sounds',
 'exterior',
 'Interiordomestic sounds',
 'animal',
 'Interiordomestic sounds',
 'Interiordomestic sounds',
 'babycry',
 'Interiordomestic sounds',
 'exterior',
 'babycry',
 'animal',
 'exterior',
 'babycry',
 'Interiordomestic sounds',
 'Interiordomestic sounds',
 'animal',
 'babycry',
 'Interiordomestic sounds',
 'babycry',
 'babycry',
 'babycry

In [81]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

X_train_T = np.array([x.reshape((128, 215, 1)) for x in X_train])
X_test_T = np.array([x.reshape((128, 215, 1)) for x in X_test])

y_train_T = np.array(keras.utils.to_categorical(label_encoder.fit_transform(np.array(y_train)),4))
y_test_T = np.array(keras.utils.to_categorical(label_encoder.fit_transform(np.array(y_test)),4))


In [88]:
from keras.layers import BatchNormalization
from keras.layers import Activation
model = Sequential()
input_shapeP=(128, 215, 1)
model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shapeP))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(4))
model.add(Activation('softmax'))

In [89]:
model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])

In [90]:
model.fit(X_train_T,y_train_T,epochs=60,validation_data=(X_test_T,y_test_T))

Train on 1200 samples, validate on 441 samples
Epoch 1/60
1200/1200 [==============================] - 2s 1ms/step - loss: 5.2313 - accuracy: 0.3217 - val_loss: 1.4240 - val_accuracy: 0.4694
Epoch 2/60
1200/1200 [==============================] - 1s 969us/step - loss: 1.6272 - accuracy: 0.4142 - val_loss: 1.2914 - val_accuracy: 0.4422
Epoch 3/60
1200/1200 [==============================] - 1s 1ms/step - loss: 1.4191 - accuracy: 0.4258 - val_loss: 1.2898 - val_accuracy: 0.4626
Epoch 4/60
1200/1200 [==============================] - 1s 985us/step - loss: 1.3105 - accuracy: 0.4625 - val_loss: 1.2970 - val_accuracy: 0.5079
Epoch 5/60
1200/1200 [==============================] - 1s 972us/step - loss: 1.2988 - accuracy: 0.4792 - val_loss: 1.2569 - val_accuracy: 0.5215
Epoch 6/60
1200/1200 [==============================] - 1s 1ms/step - loss: 1.2042 - accuracy: 0.5033 - val_loss: 1.1815 - val_accuracy: 0.5283
Epoch 7/60
1200/1200 [==============================] - 1s 969us/step - loss: 1.160

Epoch 56/60
1200/1200 [==============================] - 1s 928us/step - loss: 0.5326 - accuracy: 0.8375 - val_loss: 1.3531 - val_accuracy: 0.6939
Epoch 57/60
1200/1200 [==============================] - 1s 920us/step - loss: 0.4727 - accuracy: 0.8467 - val_loss: 1.4297 - val_accuracy: 0.6553
Epoch 58/60
1200/1200 [==============================] - 1s 927us/step - loss: 0.3853 - accuracy: 0.8583 - val_loss: 2.0188 - val_accuracy: 0.6825
Epoch 59/60
1200/1200 [==============================] - 1s 929us/step - loss: 0.3407 - accuracy: 0.8825 - val_loss: 1.9204 - val_accuracy: 0.6893
Epoch 60/60
1200/1200 [==============================] - 1s 926us/step - loss: 0.3285 - accuracy: 0.8775 - val_loss: 2.5064 - val_accuracy: 0.7075


In [91]:
model.save('C:/Users/ppeng/Documents/AI_babyclassified/model_detecting_ver_13') 

In [92]:
model_ver1 = keras.models.load_model('C:/Users/ppeng/Documents/AI_babyclassified/model_detecting_ver_13')

In [93]:
model_ver1.evaluate(X_test_T, y_test_T, verbose=0)

[2.5063620954414074, 0.7074829936027527]

In [1]:
model_ver1.predict_classes()

NameError: name 'model_ver1' is not defined

In [ ]:
model_ver1.summary()

In [ ]:
print(X_test_T[1])
print(y_test_T[1])

In [ ]:
print(test[1])

In [ ]:
data[key]['melspec']